This is my rough notebook to process and get an overview of the dataset for my thought process, experimentation, and trial and error.

In [19]:
#importing libraries for data prep
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px


In [20]:
df = pd.read_excel('..\data\HINDALCO_1D.xlsx')

In [21]:
df.head()

,datetime,close,high,low,open,volume,instrument
0,2014-01-24,114.00,115.35,113.00,113.15,5737135,HINDALCO
1,2014-01-27,111.10,112.70,109.30,112.00,8724577,HINDALCO
2,2014-01-28,113.80,115.00,109.75,110.00,4513345,HINDALCO
3,2014-01-29,111.75,114.75,111.15,114.50,4713458,HINDALCO
4,2014-01-30,108.10,110.70,107.60,110.20,5077231,HINDALCO


In [22]:
df.isnull().sum()

datetime      0
close         0
high          0
low           0
open          0
volume        0
instrument    0
dtype: int64

In [23]:
df.dtypes

datetime      datetime64[ns]
close                float64
high                 float64
low                  float64
open                 float64
volume                 int64
instrument            object
dtype: object

The datatypes are in the right format

In [24]:
df.shape

(1215, 7)

Exploring dataset

In [25]:

df['day']= df['datetime'].dt.day

In [26]:
df.set_index('datetime',inplace=True)

In [9]:

df.head()

,close,high,low,open,volume,instrument,day
datetime,,,,,,,
2014-01-24,114.00,115.35,113.00,113.15,5737135,HINDALCO,24
2014-01-27,111.10,112.70,109.30,112.00,8724577,HINDALCO,27
2014-01-28,113.80,115.00,109.75,110.00,4513345,HINDALCO,28
2014-01-29,111.75,114.75,111.15,114.50,4713458,HINDALCO,29
2014-01-30,108.10,110.70,107.60,110.20,5077231,HINDALCO,30


In [28]:
df.head()

,close,high,low,open,volume,instrument,day,8_day_MA,21_day_MA
datetime,,,,,,,,,
2014-01-24,114.00,115.35,113.00,113.15,5737135,HINDALCO,24,NaN,NaN
2014-01-27,111.10,112.70,109.30,112.00,8724577,HINDALCO,27,NaN,NaN
2014-01-28,113.80,115.00,109.75,110.00,4513345,HINDALCO,28,NaN,NaN
2014-01-29,111.75,114.75,111.15,114.50,4713458,HINDALCO,29,NaN,NaN
2014-01-30,108.10,110.70,107.60,110.20,5077231,HINDALCO,30,NaN,NaN


In [36]:
fig = px.line(df, x=df.index, y=['close', '8_day_MA', '21_day_MA'], title='Hindalco Closing price movement')
fig.show()

In [38]:
def moving_average_crossover(data, short_window =8, high_window=21):
    MA_8 = df['close'].rolling(short_window).mean()
    MA_21 = df['close'].rolling(window=high_window).mean()

    #creating a bullish crossover
    bullish_crossover = (MA_8 > MA_21) & (MA_8.shift(1) <= MA_21.shift(1))
    data.loc[bullish_crossover, 'signal'] = 'buy'

    # Create a column that signals a bearish crossover (short moving average crosses below long moving average)
    bearish_crossover = (MA_8 < MA_21) & (MA_8.shift(1) >= MA_21.shift(1))
    data.loc[bearish_crossover, 'signal'] = 'sell'

    return data

In [44]:
import plotly.graph_objects as go

# Apply the moving average crossover function
data = moving_average_crossover(df)

# Plot the stock price with the moving averages and signals
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['close'], mode='lines', name='Closing price'))
fig.add_trace(go.Scatter(x=data.index, y=data['close'].rolling(window=8).mean(), mode='lines', name='8-day MA'))
fig.add_trace(go.Scatter(x=data.index, y=data['close'].rolling(window=21).mean(), mode='lines', name='21-day MA'))
fig.add_trace(go.Scatter(x=data[data['signal']=='buy'].index, y=data[data['signal']=='buy']['close'], mode='markers', name='Buy', marker=dict(color='green', size=10)))
fig.add_trace(go.Scatter(x=data[data['signal']=='sell'].index, y=data[data['signal']=='sell']['close'], mode='markers', name='Sell', marker=dict(color='red', size=10)))

fig.update_layout(title='Moving Average Crossover', xaxis_title='Date', yaxis_title='Price')
fig.show()

In [46]:
#! pip install pandas-datareader

TypeError: 'module' object is not callable